## Setup

In [ ]:
import pandas as pd
from tensorflow import keras as ks
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Input, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping 
import matplotlib.pyplot as plt
from keras.models import load_model
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams

# Hack in order to make dictionary entries accessible as attributes (e.g. parameters.parameter)
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 18, 8
ks.backend.clear_session()

## Dataset and preprocessing

In [ ]:
# read dataset to pandas DataFrame
dataset = pd.read_table(
    #'/kaggle/input/badetemperaturer/badetemperaturer.csv', 
    #'/kaggle/input/korsvika/korsvika_water_air_hum.csv',
    '/kaggle/input/korsvika-multivariate/interpolated_data-duplicates_removed.csv',
    delimiter=",", 
    decimal=".", 
    dtype=np.float64, 
    parse_dates=["time"],
    index_col="time"
)

# For inspiration:
# https://www.tensorflow.org/tutorials/structured_data/time_series#feature_engineering

# TODO: add timeSTAMP
# TODO: see https://stackoverflow.com/questions/40881876/python-pandas-convert-datetime-to-timestamp-effectively-through-dt-accessor ??
dataset["timestamp"] = dataset.index.values.astype("int64") / 1e9
dataset["hour"] = np.sin(2*np.pi * dataset["timestamp"]/(24*60*60)) # sine-like signal
dataset["year"] = np.sin(2*np.pi * dataset["timestamp"]/(365*24*60*60))

# Engineer wind
dataset["windx"] = dataset["wind_speed"] * np.cos(2*np.pi * dataset["wind_direction"]/360)
dataset["windy"] = dataset["wind_speed"] * np.cos(2*np.pi * dataset["wind_direction"]/360)

# Remove indoor winter warnings
dataset = dataset[:10081]

# Use data from summer season only
dataset = dataset[dataset.index.month.isin([4,5,6,7,8,9,10])]

# Print dataset
print(dataset)

# Split dataset into 2019 and 2020 seasons
index_between_years = 1
while index_between_years < dataset.shape[0] and dataset.index[index_between_years].year == dataset.index[index_between_years-1].year: 
    index_between_years += 1    
dataset2019 = dataset[:index_between_years]
dataset2020 = dataset[index_between_years:]

In [ ]:
# Plot all data over half a month
dataset_sample = dataset.iloc[:72]

#sns.lineplot(x=dataset_sample.index, y="relative_humidity", data=dataset_sample);
#sns.lineplot(x=dataset_sample.index, y="rainfall", data=dataset_sample);
#sns.lineplot(x=dataset_sample.index, y="wind_speed", data=dataset_sample);
#sns.lineplot(x=dataset_sample.index, y="wind_direction", data=dataset_sample);
#sns.lineplot(x=dataset_sample.index, y="tide_obs", data=dataset_sample);
#sns.lineplot(x=dataset_sample.index, y="tide_pred", data=dataset_sample);
sns.lineplot(x=dataset_sample.index, y="water_temperature", data=dataset_sample);
sns.lineplot(x=dataset_sample.index, y="air_temperature", data=dataset_sample);
sns.lineplot(x=dataset_sample.index, y="hour", data=dataset_sample);
sns.lineplot(x=dataset_sample.index, y="year", data=dataset_sample);

In [ ]:
sns.lineplot(x=dataset2019.index, y="water_temperature", data=dataset2019);
sns.lineplot(x=dataset2019.index, y="air_temperature", data=dataset2019);

In [ ]:
sns.lineplot(x=dataset2020.index, y="water_temperature", data=dataset2020);
sns.lineplot(x=dataset2020.index, y="air_temperature", data=dataset2020);

In [ ]:
# features: "air_temperature", "relative_humidity", "rainfall", "wind_speed", "wind_direction", "tide_obs", "tide_pred", "hour"
hyperparams = AttrDict({
    "num_of_epochs": 50,
    "learning_rate": 0.004,
    "batch_size": 20,
    "loss_function": "mae",
    "earlystopping_patience": 8,
    "look_back": 15,
    "look_back_skip": 1,
    "optimizer": Adam,
    "features": ["air_temperature", "hour", "relative_humidity", "year", "windx", "windy"],
})

In [ ]:
trainx = dataset2019[hyperparams.features].values
trainy = dataset2019["water_temperature"].values

validx = dataset2020[hyperparams.features].values
validy = dataset2020["water_temperature"].values

train_time_plot = dataset2019.index.values
valid_time_plot = dataset2020.index.values

# Convert using sliding window, like in https://datascience.stackexchange.com/questions/27533/keras-lstm-with-1d-time-series/27535#27535
def step_series(x, y, t, window_size, skip=1):
    x_stepped = np.empty((x.shape[0]-window_size*skip, window_size, x.shape[1]))
    y_stepped = np.empty(x.shape[0]-window_size*skip)
    t_stepped = np.empty(x.shape[0]-window_size*skip)
    for i in range(0, x.shape[0]-window_size*skip):
        for j in range(0, window_size):
            x_stepped[i,j,:] = x[i+j*skip]
        y_stepped[i] = y[i+window_size*skip]
        t_stepped[i] = t[i+window_size*skip]
    return x_stepped, y_stepped, t_stepped

trainx_stepped, trainy_stepped, train_time_plot_stepped = step_series(trainx, trainy, train_time_plot, hyperparams.look_back, hyperparams.look_back_skip)
validx_stepped, validy_stepped, valid_time_plot_stepped = step_series(validx, validy, valid_time_plot, hyperparams.look_back, hyperparams.look_back_skip)



In [ ]:
model = Sequential()
model.add(LSTM(32, activation="swish", return_sequences=True, input_shape=(trainx_stepped.shape[1], trainx_stepped.shape[2])))
model.add(Dropout(0.05))
model.add(LSTM(32, activation="swish", return_sequences=True))
model.add(Dense(1))
model.compile(loss=hyperparams.loss_function, optimizer=hyperparams.optimizer(learning_rate=(hyperparams.learning_rate)))
model.summary()

earlystopping = EarlyStopping(monitor ="val_loss", mode ="min", patience=hyperparams.earlystopping_patience, restore_best_weights = True) 

history = model.fit(
    trainx_stepped, 
    trainy_stepped, 
    epochs=hyperparams.num_of_epochs, 
    batch_size=hyperparams.batch_size, 
    callbacks=[earlystopping], 
    validation_data=(validx_stepped, validy_stepped), 
    shuffle=False
)

In [ ]:
validx_stepped.shape

In [ ]:
plt.plot(history.history["loss"], label="(training) loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.legend()
plt.show()

In [ ]:
predy_stepped = model.predict(validx_stepped)
plt.plot(valid_time_plot_stepped, predy_stepped[:,-1,0], label="predicted")
plt.plot(valid_time_plot_stepped, validy_stepped, label="data")
plt.legend()
plt.show()

In [ ]:
model.save("output/model")